In [1]:
import pandas as pd

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

# kiwi

In [10]:
kiwi = Kiwi(typos='basic') # Kiwi 객체 생성, 기본 오타 교정 설정

In [22]:
def preprocess_hangul(my_str, keyword):
    N_list = ["NNG", "NNP"] # 수집할 명사 태그 목록
    stopwords = Stopwords() # 불용어 사전 생성
  
    keyword = kiwi.tokenize(keyword, normalize_coda=True) # 키워드를 형태소 분석 및 표준화
    # 키워드의 형태소 중 명사(NNG, NNP)인 경우 불용어 사전에 추가
    for key in keyword:
        if key.tag in N_list:
            stopwords.add((key.form, key.tag))
            
    my_str = my_str.replace('\n','') # 입력 문자열에서 줄바꿈 문자 제거
    my_str = kiwi.space(my_str)
    # 입력 문자열을 형태소 분석 및 표준화, 불용어 제거
    results = kiwi.tokenize(my_str, normalize_coda=True, stopwords=stopwords)
    
    noun_list = [] # 명사를 저장할 리스트 생성
    
    # 형태소 분석 결과에서 명사(NNG, NNP)만 추출하여 리스트에 추가
    for token in results:
        if token.tag in N_list:
            noun_list.append(token.form)
        
    return noun_list  # 명사 리스트 반환

In [25]:
files = ['네이버.csv', '카카오.csv', '라인.csv', '쿠팡.csv', '배달의민족.csv', '당근마켓.csv', '토스.csv', '직방.csv', '야놀자.csv', '삼성전자.csv', 'SK하이닉스.csv', 'DB하이텍.csv']

for file in files:
    try: # UTF-8 인코딩으로 CSV 파일 읽기 시도
        df = pd.read_csv('csv_com/'+file, encoding='utf-8')
    except UnicodeDecodeError:
        try: # UTF-8 인코딩이 실패하면 CP949 인코딩으로 읽기 시도
            df = pd.read_csv('csv_com/'+file, encoding='cp949')
        except UnicodeDecodeError: # CP949 인코딩이 실패하면 EUC-KR 인코딩으로 읽기 시도
            df = pd.read_csv('csv_com/'+file, encoding='euc-kr')

    keyword = file.split(".")[0] # 파일명에서 키워드 추출 (확장자 제외)
    
    # 'content' 열을 전처리하여 'preprocessed_content' 열에 저장
    df['preprocessed_content'] = df['content'].apply(lambda x: preprocess_hangul(x, keyword))
    # 전처리된 데이터를 새로운 CSV 파일로 저장
    df.to_csv('csv_pre/{}_전처리.csv'.format(keyword), encoding='utf-8-sig', index=False)

In [ ]:
keywords = ['네이버', '카카오', '라인', '쿠팡', '배달의민족', '당근마켓', '토스', '직방', '야놀자', '삼성전자', 'SK하이닉스', 'DB하이텍']

for keyword in keywords:
    # 키워드에 해당하는 뉴스와 블로그 파일 리스트 생성
    files = [f'{keyword}_뉴스.csv', f'{keyword}_블로그.csv']
    
    for file in files:
        try: # UTF-8 인코딩으로 CSV 파일 읽기 시도
            df = pd.read_csv('csv/'+file, encoding='utf-8')
        except UnicodeDecodeError:
            try: # UTF-8 인코딩이 실패하면 CP949 인코딩으로 읽기 시도
                df = pd.read_csv('csv/'+file, encoding='cp949')
            except UnicodeDecodeError: # CP949 인코딩이 실패하면 EUC-KR 인코딩으로 읽기 시도
                df = pd.read_csv('csv/'+file, encoding='euc-kr')

        filename = file.split(".")[0] # 파일명에서 확장자를 제외한 이름 추출
        
        # 'content' 열을 전처리하여 'preprocessed_content' 열에 저장
        df['preprocessed_content'] = df['content'].apply(lambda x: preprocess_hangul(x, keyword))
        # 전처리된 데이터를 새로운 CSV 파일로 저장
        df.to_csv('csv_pre/{}_전처리.csv'.format(filename), encoding='utf-8-sig', index=False)
